In [1]:
!which python; python -V;
from astropy import units as u

from perylune.orbit_tools import *
from perylune.interplanetary import *
from perylune.mpc import *

import time

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


In [2]:
# First step is to load the orbit data. parse_txt() will load the orbital elements from text file in MPC format.
# This will be returned as an array. Each entry in that array has a list of 8 parameters or so.
elements = parse_txt("../data/mpc/mpcorb_extended.dat", limit=0, skip="------------------")

print("Number of orbital elements parsed: %d" % len(elements))

Read 1028498 line(s) from NEA.txt, parsing up to 0 line(s)
Number of orbital elements parsed: 1028453


In [3]:
# Now process the data. Find min and max a and e, also put the a and e values in their separate arrays (a_tbl, e_tbl)

max_e = 0
min_e = 9999999
max_a = 0
min_a = 9999999

a_tbl = []
e_tbl = []
cnt = 0
for e in elements:
    a = e[8].value
    ecc = e[6].value
    if a < 6:
        cnt = cnt + 1
        a_tbl.append(a)
        e_tbl.append(e)
    if max_a < a:
        max_a = a
    if min_a > a:
        min_a = a
    if max_e < ecc:
        max_e = ecc
    if min_e > ecc:
        min_e = ecc

print ("Processed %d asteroids, a=%f..%f [AU], eccentricity = %f..%f" % (cnt, min_a, max_a, min_e, max_e))

Processed 1024421 asteroids, a=0.555412..3549.257149 [AU], eccentricity = 0.000000..0.998747


In [4]:
# This is the major processing part (that sadly takes a lot of time to complete). It's possible to limit
# the number of asteroids processed by setting max to some value e.g. 20000. However, this will process
# the first 20000 lines from the input file and they're in random (or historical detection) order.
#
# The result is stored in dv_tbl array.
max = 2000000
t0 = time.time()
i = 0

dv_tbl = []
print("Starting calculation, time %s" % time.time())
for e in elements:
    dv_burn1, dv_burn2, dv, tof = calc_min_delta_v(1.0*u.au, e)
    dv_tbl.append([e[0], dv, tof, dv_burn1, dv_burn2])
    i = i + 1
    if (i % 10000 == 0):
        print("Calculated %d orbits, time %s" % (i, time.time()))
    if i >= max:
        break
t1 = time.time()

print("Calculated for %d orbits, calculations took %s s." % (i, (t1-t0)))

Starting calculation, time 1605138454.17286
Calculated 10000 orbits, time 1605138461.25387
Calculated 20000 orbits, time 1605138468.2517946
Calculated 30000 orbits, time 1605138475.2271688
Calculated 40000 orbits, time 1605138482.2088675
Calculated 50000 orbits, time 1605138489.205532
Calculated 60000 orbits, time 1605138496.2299635
Calculated 70000 orbits, time 1605138504.975626
Calculated 80000 orbits, time 1605138512.0324328
Calculated 90000 orbits, time 1605138519.0578794
Calculated 100000 orbits, time 1605138526.0392518
Calculated 110000 orbits, time 1605138533.0169706
Calculated 120000 orbits, time 1605138540.2507865
Calculated 130000 orbits, time 1605138547.542095
Calculated 140000 orbits, time 1605138554.78781
Calculated 150000 orbits, time 1605138561.9948137
Calculated 160000 orbits, time 1605138569.1886222
Calculated 170000 orbits, time 1605138576.3872247
Calculated 180000 orbits, time 1605138583.586671
Calculated 190000 orbits, time 1605138590.7797406
Calculated 200000 orbit

In [5]:
# Now sort the table by the second (index 1) value. This is the total delta-v
t2 = time.time()
dv_tbl2 = sorted(dv_tbl, key=lambda e: e[1].value)
t3 = time.time()

print("dv_tbl (%d elements) is now sorted, sorting took %s s." % (len(dv_tbl), t3-t2))

dv_tbl (1028453 elements) is now sorted, sorting took 1.0772876739501953 s.


In [6]:
# Now print the top 10 asteroids (with smallest delta-v)
top_x = 10
print(len(dv_tbl2))
print(" TOP %d ASTEROIDS THAT ARE EASIEST TO REACH" % top_x)
for i in range(top_x):
    x = dv_tbl2[i]
    print("%d. %s:  fly-by delta-v=%s randezvous delta-v=%s" % (i+1, x[0], x[3], x[1] ))

1028453
 TOP 10 ASTEROIDS THAT ARE EASIEST TO REACH
1. 2010 FN:  fly-by delta-v=100.73845547422833 m / s randezvous delta-v=178.38592193633747 m / s
2. 2018 UC:  fly-by delta-v=132.27082396382207 m / s randezvous delta-v=189.4751242873342 m / s
3. 2016 JA:  fly-by delta-v=111.15564339177044 m / s randezvous delta-v=218.82623504831835 m / s
4. 2020 SO:  fly-by delta-v=123.50356470013689 m / s randezvous delta-v=224.53306310902377 m / s
5. 2019 YB4:  fly-by delta-v=280.1225044691703 m / s randezvous delta-v=308.8324268322133 m / s
6. 2019 AU6:  fly-by delta-v=218.92547156701437 m / s randezvous delta-v=342.59523498058877 m / s
7. 2000 SG344:  fly-by delta-v=180.04330838593847 m / s randezvous delta-v=351.4091885944977 m / s
8. 2010 JK1:  fly-by delta-v=232.52398230729133 m / s randezvous delta-v=446.5553471761266 m / s
9. 2016 WQ3:  fly-by delta-v=348.1024745741976 m / s randezvous delta-v=466.27559915444874 m / s
10. 2013 BS45:  fly-by delta-v=406.0127279657208 m / s randezvous delta-v=

In [7]:
# And write all the data to output file.
f = open("../data/mpc/asteroids.txt", "w+")
f.write("#name, total delta-v[m/s], time of flight[days], Hohmann burn1 [m/s], Hohmann burn2 [m/s]\n")
for x in dv_tbl2:
    f.write("%s, %4.2f, %4.1f, %4.2f, %4.2f\n" % (x[0], x[1].value, x[2].value, x[3].value, x[4].value))
f.close()